## USA Baby Names 1880-2016

The United States Social Security Administration maintains an interesting data set of (almost) all names given to babies born in the United States, by sex and year, going back to 1880. This data set is available at [https://www.ssa.gov/oact/babynames/limits.html](https://www.ssa.gov/oact/babynames/limits.html)

This data set is interesting and fun to explore and we'll use it as the basis of a simple data analysis project with the end goal of creating a script that can be called to output a plot of a single name's popularity over time.

To start, we will assume that this dataset has already been downloaded and unzipped.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
# Set up some variables for use later
dataset_path = 'data\\names'
begin_year = 1880
end_year = 2016

Let's first examine the data files to see what we're working with. Note the `type` command on Windows is equivalent to `cat` on MacOS or Linux.

In [ ]:
!dir $dataset_path

In [ ]:
# Read a single file into a python variable and print out the first five lines
sample = !type $dataset_path\\yob1880.txt
sample[:5]

We will need a function to read in all of these files one by one and combine them into a single dataframe.

In [ ]:
def create_dataframe():
    columns = ('name', 'sex', 'births')
    pieces = []
    for year in range(begin_year, end_year + 1):
        filename = '%s/yob%d.txt' % (dataset_path, year)
        piece = pd.read_csv(filename, names=columns)
        piece['year'] = year
        pieces.append(piece)
        
    return pd.concat(pieces, ignore_index=True)

In [ ]:
# Now call our new function to get the dataset loaded into a Dataframe.
df = create_dataframe()
df.head()

In [ ]:
# How many records do we have?
len(df)

Now that we have the data in a dataframe, we want to move the year and sex columns into the index, leaving only columns for name and birth count. We can use the `set_index` method of the dataframe for this.

In [ ]:
df = df.set_index(keys=['year', 'sex'])
df.head()

Now we need a function that, given a sex and a year, returns a series containing the number of births by year.

In [ ]:
def get_births_series(df, name, sex):
    single_sex_df = df.xs(sex, level='sex')
    return single_sex_df[single_sex_df.name == name]['births']

In [ ]:
matthews = get_births_series(df, 'Matthew', 'M')
matthews.head()

In [ ]:
plt.style.use('seaborn')
matthews.plot(title='Annual count of births for name %s' % 'Matthew')

Now one last function to output a plot of the series. Just the bare minimum for now.

In [ ]:
def create_births_figure(s, sex, name):
    plt.style.use('seaborn')
    sex_full = 'female'
    if sex == 'M':
        sex_full = 'male'
    plot = s.plot(title='Annual count of US %s births for name %s' % (sex_full, name))
    return plot.get_figure()

In [ ]:
fig = create_births_figure(matthews, 'M', 'Matthew')